<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness/66_Fraksi5_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 6 dijalankan menggunakan email shintafiaa@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [102]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-sj9sgdj9
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-sj9sgdj9
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-0otcczk4/pyfolio_f0b69f0f9d364baf8457e56bdd51ff27
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-0otcczk4/pyfolio_f0b69f0f9d364baf8457e56bdd51ff27
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-0otcczk4/elegantrl_73a35726e98b4657b55a51fecc5543af
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-0otcczk4/elegantrl_73a35726e98b4657b55a51fecc5543af


<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [103]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [104]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [105]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [106]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [107]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi5).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10912, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [108]:
df.shape

(10912, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [109]:
df.nunique()

date       2728
open       1787
high       1851
low        1822
close      4889
volume    10004
tic           4
day           5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [110]:
print(df['tic'].unique())

['INCO.JK' 'INDF.JK' 'INTP.JK' 'UNTR.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [111]:
df.describe()

,open,high,low,close,volume,day
count,10912.000000,10912.000000,10912.000000,10912.000000,1.091200e+04,10912.000000
mean,12277.784482,12443.054341,12099.235082,9214.522186,8.174067e+06,1.988636
std,8848.437416,8957.200965,8726.781880,6663.511158,1.155167e+07,1.410209
min,840.000000,850.000000,820.000000,547.314026,0.000000e+00,0.000000
25%,4200.000000,4250.000000,4125.000000,3407.211426,2.582954e+06,1.000000
50%,8100.000000,8150.000000,7975.000000,6529.755859,4.948900e+06,2.000000
75%,19500.000000,19800.000000,19250.000000,14226.301270,9.709625e+06,3.000000
max,40425.000000,40500.000000,39950.000000,33177.527344,3.460175e+08,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [112]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,2050.000000,2300.000000,2030.000000,1792.011108,30353500,INCO.JK,0
1,2009-01-05,980.000000,980.000000,920.000000,638.533020,154640500,INDF.JK,0
2,2009-01-05,4600.000000,5050.000000,4600.000000,3106.131104,3534000,INTP.JK,0
3,2009-01-05,4326.423828,5071.530273,4326.423828,3122.124268,25673281,UNTR.JK,0
4,2009-01-06,2300.000000,2325.000000,2100.000000,1733.575928,21013000,INCO.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [113]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [114]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [115]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10604.000000,10604.000000,10604.000000,10604.000000,1.060400e+04,10604.000000,10604.000000,10604.000000,10604.000000,10604.000000,10604.000000
mean,12271.640035,12438.118821,12092.935433,9207.742441,8.233058e+06,2.019238,24.972092,51.520274,10.412414,18.181388,3.853375
std,8841.779597,8951.651265,8720.044229,6656.181953,1.153550e+07,1.398187,228.574967,8.170004,112.972173,7.336007,5.556669
min,840.000000,850.000000,820.000000,547.314026,0.000000e+00,0.000000,-1449.822787,0.000000,-510.210205,9.140000,0.000000
25%,4200.000000,4250.000000,4125.000000,3407.890869,2.620000e+06,1.000000,-63.195157,46.248757,-75.284330,13.280000,0.792609
50%,8100.000000,8150.000000,7987.500000,6529.755859,4.996350e+06,2.000000,20.934628,51.561393,13.254781,15.990000,2.218943
75%,19500.000000,19800.000000,19231.250000,14215.475830,9.807825e+06,3.000000,124.484781,56.925774,95.167747,20.629999,4.864719
max,40425.000000,40500.000000,39950.000000,33177.527344,3.460175e+08,4.000000,1224.640114,100.000000,648.948808,56.650002,86.424860


In [116]:
processed.nunique()

date           2651
open           1775
high           1835
low            1809
close          4829
volume         9764
tic               4
day               5
macd          10601
rsi_30         9630
cci_30        10600
vix            1441
turbulence     2397
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [117]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['INCO.JK', 'INDF.JK', 'INTP.JK', 'UNTR.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [118]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [119]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'INCO.JK'),
 ('2009-01-05', 'INDF.JK'),
 ('2009-01-05', 'INTP.JK'),
 ('2009-01-05', 'UNTR.JK'),
 ('2009-01-06', 'INCO.JK'),
 ('2009-01-06', 'INDF.JK'),
 ('2009-01-06', 'INTP.JK'),
 ('2009-01-06', 'UNTR.JK'),
 ('2009-01-07', 'INCO.JK'),
 ('2009-01-07', 'INDF.JK'),
 ('2009-01-07', 'INTP.JK'),
 ('2009-01-07', 'UNTR.JK'),
 ('2009-01-08', 'INCO.JK'),
 ('2009-01-08', 'INDF.JK'),
 ('2009-01-08', 'INTP.JK'),
 ('2009-01-08', 'UNTR.JK'),
 ('2009-01-09', 'INCO.JK'),
 ('2009-01-09', 'INDF.JK'),
 ('2009-01-09', 'INTP.JK'),
 ('2009-01-09', 'UNTR.JK'),
 ('2009-01-10', 'INCO.JK'),
 ('2009-01-10', 'INDF.JK'),
 ('2009-01-10', 'INTP.JK'),
 ('2009-01-10', 'UNTR.JK'),
 ('2009-01-11', 'INCO.JK'),
 ('2009-01-11', 'INDF.JK'),
 ('2009-01-11', 'INTP.JK'),
 ('2009-01-11', 'UNTR.JK'),
 ('2009-01-12', 'INCO.JK'),
 ('2009-01-12', 'INDF.JK'),
 ('2009-01-12', 'INTP.JK'),
 ('2009-01-12', 'UNTR.JK'),
 ('2009-01-13', 'INCO.JK'),
 ('2009-01-13', 'INDF.JK'),
 ('2009-01-13', 'INTP.JK'),
 ('2009-01-13', 'UNT

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [120]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,UNTR.JK,22300.000000,22325.000000,21800.000000,19353.603516,3803500.0,2.0,-49.096164,50.405707,6.805590,12.580000,4.030015
16000,2019-12-19,INCO.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,INDF.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,INTP.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [121]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.290000,0.951754
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030015
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030015
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626136,49.586671,-17.159798,12.580000,4.030015


In [122]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.290000,0.951754
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030015
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030015
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626136,49.586671,-17.159798,12.580000,4.030015


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [123]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.290000,0.951754
15996,2019-12-18,INCO.JK,3600.000000,3600.000000,3440.000000,3418.002930,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030015
15997,2019-12-18,INDF.JK,7975.000000,7975.000000,7900.000000,7000.262207,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030015
15998,2019-12-18,INTP.JK,20000.000000,20075.000000,19800.000000,17098.693359,2290000.0,2.0,-13.626136,49.586671,-17.159798,12.580000,4.030015


In [124]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.000000,2300.000000,2030.000000,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
1,2009-01-05,INDF.JK,980.000000,980.000000,920.000000,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
2,2009-01-05,INTP.JK,4600.000000,5050.000000,4600.000000,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.0
4,2009-01-06,INCO.JK,2300.000000,2325.000000,2100.000000,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.0
5,2009-01-06,INDF.JK,1000.000000,1110.000000,980.000000,664.595581,346017500.0,1.0,0.584737,100.000000,66.666667,38.560001,0.0
6,2009-01-06,INTP.JK,5100.000000,5250.000000,5050.000000,3198.392578,2973000.0,1.0,2.069969,100.000000,66.666667,38.560001,0.0
7,2009-01-06,UNTR.JK,5143.637207,5335.922852,4759.066406,3048.140625,23181674.0,1.0,-1.659889,0.000000,66.666667,38.560001,0.0
8,2009-01-07,INCO.JK,2300.000000,2650.000000,2275.000000,2025.751587,58399500.0,2.0,7.389817,83.798871,100.000000,43.389999,0.0
9,2009-01-07,INDF.JK,1020.000000,1080.000000,1020.000000,664.595581,147142500.0,2.0,0.745071,100.000000,53.392705,43.389999,0.0


In [125]:
processed_full.nunique()

date           2651
tic               4
open           1775
high           1835
low            1809
close          4829
volume         9764
day               5
macd          10601
rsi_30         9630
cci_30        10600
vix            1441
turbulence     2397
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [126]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10604 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10604 non-null  object 
 1   tic         10604 non-null  object 
 2   open        10604 non-null  float64
 3   high        10604 non-null  float64
 4   low         10604 non-null  float64
 5   close       10604 non-null  float64
 6   volume      10604 non-null  float64
 7   day         10604 non-null  float64
 8   macd        10604 non-null  float64
 9   rsi_30      10604 non-null  float64
 10  cci_30      10604 non-null  float64
 11  vix         10604 non-null  float64
 12  turbulence  10604 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [127]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi5/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2050.0,2300.0,2030.0,1792.011108,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2300.0,2325.0,2100.0,1733.575928,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
8,2009-01-07,INCO.JK,2300.0,2650.0,2275.0,2025.751587,58399500.0,2.0,7.389817,83.798871,100.000000,43.389999,0.000000
12,2009-01-08,INCO.JK,2525.0,2575.0,2400.0,1967.316406,26228000.0,3.0,9.056255,71.416827,65.969176,42.560001,0.000000
16,2009-01-09,INCO.JK,2525.0,2825.0,2525.0,2181.578857,21530000.0,4.0,18.297111,81.682923,109.658453,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.940000,5.727264
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.630000,4.966053
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.140000,5.562965
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.290000,0.951754


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,INDF.JK,980.0,980.0,920.0,638.533020,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
5,2009-01-06,INDF.JK,1000.0,1110.0,980.0,664.595581,346017500.0,1.0,0.584737,100.000000,66.666667,38.560001,0.000000
9,2009-01-07,INDF.JK,1020.0,1080.0,1020.0,664.595581,147142500.0,2.0,0.745071,100.000000,53.392705,43.389999,0.000000
13,2009-01-08,INDF.JK,1030.0,1050.0,990.0,671.111206,23846500.0,3.0,1.034978,100.000000,16.400202,42.560001,0.000000
17,2009-01-09,INDF.JK,1030.0,1110.0,1020.0,710.205078,144413500.0,4.0,2.726356,100.000000,101.730326,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.940000,5.727264
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.630000,4.966053
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.140000,5.562965
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.290000,0.951754


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,INTP.JK,4600.0,5050.0,4600.0,3106.131104,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
6,2009-01-06,INTP.JK,5100.0,5250.0,5050.0,3198.392578,2973000.0,1.0,2.069969,100.000000,66.666667,38.560001,0.000000
10,2009-01-07,INTP.JK,5000.0,5250.0,4900.0,3075.378174,3731500.0,2.0,-1.177118,42.029163,16.197382,43.389999,0.000000
14,2009-01-08,INTP.JK,4500.0,5100.0,4500.0,3136.885010,530000.0,3.0,-0.404428,55.401812,-68.751685,42.560001,0.000000
18,2009-01-09,INTP.JK,4800.0,5000.0,4800.0,2952.362549,549500.0,4.0,-7.318619,32.287176,-54.671858,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.940000,5.727264
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.630000,4.966053
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.140000,5.562965
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.290000,0.951754


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,UNTR.JK,4326.423828,5071.530273,4326.423828,3122.124268,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
7,2009-01-06,UNTR.JK,5143.637207,5335.922852,4759.066406,3048.140625,23181674.0,1.0,-1.659889,0.000000,66.666667,38.560001,0.000000
11,2009-01-07,UNTR.JK,5239.779785,5576.279785,4951.351562,3225.702637,22293412.0,2.0,3.391159,71.287275,99.307905,43.389999,0.000000
15,2009-01-08,UNTR.JK,4999.423340,5239.779785,4855.208984,3077.733887,6885594.0,3.0,0.263010,44.153108,5.364895,42.560001,0.000000
19,2009-01-09,UNTR.JK,5191.708496,5239.779785,4951.351562,3196.108887,7648522.0,4.0,3.268098,57.531049,42.654740,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.000000,20875.000000,20450.000000,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.940000,5.727264
15979,2019-12-13,UNTR.JK,20750.000000,21200.000000,20700.000000,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.630000,4.966053
15991,2019-12-16,UNTR.JK,21250.000000,22125.000000,21200.000000,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.140000,5.562965
15995,2019-12-17,UNTR.JK,22000.000000,22375.000000,21750.000000,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.290000,0.951754


In [128]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,INCO.JK,2.050000e+05,2.300000e+05,2.030000e+05,1.792011e+05,30353500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,INDF.JK,9.800000e+04,9.800000e+04,9.200000e+04,6.385330e+04,154640500.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,INTP.JK,4.600000e+05,5.050000e+05,4.600000e+05,3.106131e+05,3534000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,UNTR.JK,4.326424e+05,5.071530e+05,4.326424e+05,3.122124e+05,25673281.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,INCO.JK,2.300000e+05,2.325000e+05,2.100000e+05,1.733576e+05,21013000.0,1.0,-1.311046,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNTR.JK,2.200000e+06,2.237500e+06,2.175000e+06,1.986408e+06,4607100.0,1.0,-78.720954,52.760264,20.354842,12.290000,0.951754
15996,2019-12-18,INCO.JK,3.600000e+05,3.600000e+05,3.440000e+05,3.418003e+05,14110300.0,2.0,27.673484,51.602337,90.205381,12.580000,4.030015
15997,2019-12-18,INDF.JK,7.975000e+05,7.975000e+05,7.900000e+05,7.000262e+05,8303900.0,2.0,11.639505,53.440010,59.891878,12.580000,4.030015
15998,2019-12-18,INTP.JK,2.000000e+06,2.007500e+06,1.980000e+06,1.709869e+06,2290000.0,2.0,-13.626136,49.586671,-17.159798,12.580000,4.030015


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [129]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1976


In [130]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,UNTR.JK,3457500.0,3460000.0,3422500.0,2.839685e+06,2636700.0,3.0,297.519672,55.675875,163.784499,10.18,5.337388
2156,2017-12-29,INCO.JK,285000.0,291000.0,285000.0,2.863197e+05,9727700.0,4.0,-6.167929,51.919947,61.612511,11.04,3.007390
2156,2017-12-29,INDF.JK,770000.0,777500.0,760000.0,6.258995e+05,12848100.0,4.0,-15.327443,44.909583,53.529862,11.04,3.007390
2156,2017-12-29,INTP.JK,2150000.0,2195000.0,2150000.0,1.754518e+06,1534000.0,4.0,288.716889,58.947146,197.982418,11.04,3.007390
2156,2017-12-29,UNTR.JK,3460000.0,3540000.0,3457500.0,2.905342e+06,4163100.0,4.0,377.482385,58.036908,202.252623,11.04,3.007390


In [131]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,INCO.JK,292000.0,304000.0,292000.0,3.001898e+05,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092
0,2018-01-02,INDF.JK,765000.0,770000.0,755000.0,6.197431e+05,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092
0,2018-01-02,INTP.JK,2195000.0,2300000.0,2135000.0,1.838447e+06,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092
0,2018-01-02,UNTR.JK,3540000.0,3540000.0,3410000.0,2.802752e+06,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092
1,2018-01-03,INCO.JK,305000.0,311000.0,304000.0,3.021713e+05,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817


In [132]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [133]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [134]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [135]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [136]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [137]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 361       |
|    iterations         | 100       |
|    time_elapsed       | 41        |
|    total_timesteps    | 15000     |
| train/                |           |
|    entropy_loss       | -5.68     |
|    explained_variance | -0.00542  |
|    learning_rate      | 0.0014    |
|    n_updates          | 99        |
|    policy_loss        | -1.33e+03 |
|    reward             | -86.91571 |
|    std                | 1         |
|    value_loss         | 1.36e+05  |
-------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 164944972.19
total_reward: 64944972.19
total_cost: 774169.20
total_trades: 4407
Sharpe: 0.418
--------------------------------------
| time/                 |            |
|    fps                | 366        |
|    iterations         | 200        |
|    time_elapsed       | 81         |
|    total_timesteps    | 30000

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [138]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [139]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 235055.32
total_reward: -99764944.68
total_cost: 29160321.03
total_trades: 5264
Sharpe: -1.133
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 400100.80
total_reward: -99599899.20
total_cost: 44541610.83
total_trades: 5341
Sharpe: -1.071
----------------------------------
| time/              |           |
|    fps             | 376       |
|    iterations      | 1         |
|    time_elapsed    | 132       |
|    total_timesteps | 50000     |
| train/             |           |
|    reward          | 22.755098 |
----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [140]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [141]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 704828245.11
total_reward: 604828245.11
total_cost: 11847481.31
total_trades: 2424
Sharpe: 0.830
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 97        |
|    time_elapsed    | 88        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.56e+07  |
|    critic_loss     | 1.76e+18  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | -330.6011 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 86        |
|    time_elapsed    | 199       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 4.05e+07  |
|    critic_loss     | 1.38e+16  |
|    learning_rate   | 0.9       |
|    n_updates       | 150

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [142]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [143]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 67        |
|    time_elapsed    | 127       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.53e+04  |
|    critic_loss     | 4.05e+05  |
|    ent_coef        | 1.25      |
|    learning_rate   | 0.0001    |
|    n_updates       | 8613      |
|    reward          | 242.93246 |
----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 107572282.83
total_reward: 7572282.83
total_cost: 229300.96
total_trades: 4310
Sharpe: 0.476
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 67        |
|    time_elapsed    | 257       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 2.23e+04  |
|    critic_loss     | 4.42e+05  |
|    ent_coef        | 1.25   

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [144]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [145]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 157963102.10
total_reward: 57963102.10
total_cost: 229389.04
total_trades: 4312
Sharpe: 0.397
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 129        |
|    time_elapsed    | 66         |
|    total_timesteps | 8628       |
| train/             |            |
|    actor_loss      | 2.46e+04   |
|    critic_loss     | 2.64e+07   |
|    learning_rate   | 0.0003     |
|    n_updates       | 6471       |
|    reward          | -58.972233 |
-----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 115        |
|    time_elapsed    | 149        |
|    total_timesteps | 17256      |
| train/             |            |
|    actor_loss      | 1.06e+04   |
|    critic_loss     | 4.38e+05   |
|    learning_rate   | 0.0003     |
|    n

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [146]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [147]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [148]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [149]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.740824
std         5.794682
min         0.000000
25%         0.684348
50%         2.070520
75%         4.635445
max        86.424860
Name: turbulence, dtype: float64

In [150]:
insample_risk_indicator.turbulence.quantile(0.996)

39.04862070256753

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [151]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [152]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,INCO.JK,292000.0,304000.0,292000.0,3.001898e+05,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092
0,2018-01-02,INDF.JK,765000.0,770000.0,755000.0,6.197431e+05,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092
0,2018-01-02,INTP.JK,2195000.0,2300000.0,2135000.0,1.838447e+06,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092
0,2018-01-02,UNTR.JK,3540000.0,3540000.0,3410000.0,2.802752e+06,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092
1,2018-01-03,INCO.JK,305000.0,311000.0,304000.0,3.021713e+05,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [153]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [154]:
df_actions_and_account_value_a2c = pd.merge(df_actions_a2c, df_account_value_a2c, on='date', how='outer')

#adding new column
df_actions_and_account_value_a2c['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_a2c.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_a2c = df_actions_and_account_value_a2c.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_a2c['total_shares'] = action_columns_a2c.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_a2c['total_shares'] = df_actions_and_account_value_a2c['total_shares'].cumsum(skipna=False)

In [155]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#swap points and rebounds columns
df_actions_and_account_value_a2c = swap_columns(df_actions_and_account_value_a2c, 'total_shares','account_value')

#Replace NaN value to 0
df_actions_and_account_value_a2c=df_actions_and_account_value_a2c.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_a2c)

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,0.0,12.0,0.0,1.0,13.0,1.000000e+08
1,2018-01-03,0.0,12.0,0.0,1.0,26.0,9.998672e+07
2,2018-01-04,0.0,12.0,0.0,1.0,39.0,1.002627e+08
3,2018-01-05,0.0,12.0,0.0,1.0,52.0,1.004419e+08
4,2018-01-08,0.0,-15.0,0.0,-4.0,33.0,1.011976e+08
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,1.0,0.0,0.0,77.0,9.083423e+07
490,2019-12-13,4.0,-22.0,0.0,0.0,59.0,9.168021e+07
491,2019-12-16,-4.0,1.0,0.0,0.0,56.0,9.224888e+07
492,2019-12-17,0.0,1.0,0.0,0.0,57.0,9.260938e+07


In [156]:
df_actions_and_account_value_a2c.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi5/df_actions_and_account_value_a2c(4).csv',index=False)

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [157]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [158]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [159]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,0.0,0.0,0.0,100000000.0
1,2018-01-03,0.0,0.0,0.0,0.0,0.0,100000000.0
2,2018-01-04,0.0,0.0,0.0,0.0,0.0,100000000.0
3,2018-01-05,0.0,0.0,0.0,0.0,0.0,100000000.0
4,2018-01-08,0.0,0.0,0.0,0.0,0.0,100000000.0
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,0.0,0.0,0.0,0.0,100000000.0
490,2019-12-13,0.0,0.0,0.0,0.0,0.0,100000000.0
491,2019-12-16,0.0,0.0,0.0,0.0,0.0,100000000.0
492,2019-12-17,0.0,0.0,0.0,0.0,0.0,100000000.0


In [160]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi5/df_actions_and_account_value_ppo(4).csv',index=False)

<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [161]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


In [162]:
df_actions_and_account_value_td3 = pd.merge(df_actions_td3, df_account_value_td3, on='date', how='outer')

#adding new column
df_actions_and_account_value_td3['total_shares'] = ""

#selecting column of actions in every tics
action_columns_td3 = df_actions_and_account_value_td3.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_td3['total_shares'] = action_columns_td3.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_td3['total_shares'] = df_actions_and_account_value_td3['total_shares'].cumsum(skipna=False)

In [163]:
#swap points and rebounds columns
df_actions_and_account_value_td3 = swap_columns(df_actions_and_account_value_td3, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_td3=df_actions_and_account_value_td3.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_td3)

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,100.0,100.0,0.0,0.0,200.0,1.000000e+08
1,2018-01-03,0.0,12.0,0.0,0.0,212.0,1.008074e+08
2,2018-01-04,-100.0,48.0,0.0,0.0,160.0,1.020740e+08
3,2018-01-05,0.0,0.0,0.0,0.0,160.0,1.011810e+08
4,2018-01-08,0.0,0.0,0.0,0.0,160.0,1.034794e+08
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,0.0,0.0,0.0,160.0,1.078058e+08
490,2019-12-13,0.0,0.0,0.0,0.0,160.0,1.095668e+08
491,2019-12-16,0.0,0.0,0.0,0.0,160.0,1.113279e+08
492,2019-12-17,0.0,0.0,0.0,0.0,160.0,1.123845e+08


In [164]:
df_actions_and_account_value_td3.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi5/df_actions_and_account_value_td3(4).csv',index=False)

<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [165]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [166]:
df_actions_and_account_value_sac = pd.merge(df_actions_sac, df_account_value_sac, on='date', how='outer')

#adding new column
df_actions_and_account_value_sac['total_shares'] = ""

#selecting column of actions in every tics
action_columns_sac = df_actions_and_account_value_sac.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_sac['total_shares'] = action_columns_sac.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_sac['total_shares'] = df_actions_and_account_value_sac['total_shares'].cumsum(skipna=False)

In [167]:
#swap points and rebounds columns
df_actions_and_account_value_sac = swap_columns(df_actions_and_account_value_sac, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_sac=df_actions_and_account_value_sac.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_sac)

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,0.0,35.0,35.0,1.000000e+08
1,2018-01-03,5.0,0.0,0.0,0.0,40.0,9.668643e+07
2,2018-01-04,0.0,0.0,0.0,0.0,40.0,9.936975e+07
3,2018-01-05,0.0,0.0,0.0,0.0,40.0,1.034928e+08
4,2018-01-08,0.0,0.0,0.0,0.0,40.0,1.042827e+08
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,0.0,0.0,0.0,40.0,6.537100e+07
490,2019-12-13,0.0,0.0,0.0,0.0,40.0,6.768814e+07
491,2019-12-16,0.0,0.0,0.0,0.0,40.0,7.031104e+07
492,2019-12-17,0.0,0.0,0.0,0.0,40.0,7.147625e+07


In [168]:
df_actions_and_account_value_sac.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi5/df_actions_and_account_value_sac(4).csv',index=False)

<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [169]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


In [170]:
df_actions_and_account_value_ddpg = pd.merge(df_actions_ddpg, df_account_value_ddpg, on='date', how='outer')

#adding new column
df_actions_and_account_value_ddpg['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ddpg = df_actions_and_account_value_ddpg.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ddpg['total_shares'] = action_columns_ddpg.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ddpg['total_shares'] = df_actions_and_account_value_ddpg['total_shares'].cumsum(skipna=False)

In [171]:
#swap points and rebounds columns
df_actions_and_account_value_ddpg = swap_columns(df_actions_and_account_value_ddpg, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ddpg=df_actions_and_account_value_ddpg.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ddpg)

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,100.0,100.0,0.0,0.0,200.0,1.000000e+08
1,2018-01-03,0.0,12.0,0.0,0.0,212.0,1.008074e+08
2,2018-01-04,0.0,0.0,0.0,0.0,212.0,1.020740e+08
3,2018-01-05,0.0,0.0,0.0,0.0,212.0,1.022088e+08
4,2018-01-08,0.0,0.0,0.0,0.0,212.0,1.038177e+08
...,...,...,...,...,...,...,...
489,2019-12-12,0.0,0.0,0.0,0.0,212.0,1.087285e+08
490,2019-12-13,0.0,0.0,0.0,0.0,212.0,1.112492e+08
491,2019-12-16,0.0,0.0,0.0,0.0,212.0,1.136708e+08
492,2019-12-17,0.0,0.0,0.0,0.0,212.0,1.144105e+08


In [172]:
df_actions_and_account_value_ddpg.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi5/df_actions_and_account_value_ddpg(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.1. Agen 1: A2C

In [173]:
#Replace NaN value to 0
action_columns_a2c=action_columns_a2c.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_a2c)

j=0
for j in range(numberOfRows):
  for i in action_columns_a2c:
   val = action_columns_a2c[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_a2c, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi5/df_actions_and_account_str_value_a2c(4).csv',index=False)
df_actions_str

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 12.0 saham,Tahan,Beli 1.0 saham,13.0,1.000000e+08
1,2018-01-03,Tahan,Beli 12.0 saham,Tahan,Beli 1.0 saham,26.0,9.998672e+07
2,2018-01-04,Tahan,Beli 12.0 saham,Tahan,Beli 1.0 saham,39.0,1.002627e+08
3,2018-01-05,Tahan,Beli 12.0 saham,Tahan,Beli 1.0 saham,52.0,1.004419e+08
4,2018-01-08,Tahan,Jual -15.0 saham,Tahan,Jual -4.0 saham,33.0,1.011976e+08
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Beli 1.0 saham,Tahan,Tahan,77.0,9.083423e+07
490,2019-12-13,Beli 4.0 saham,Jual -22.0 saham,Tahan,Tahan,59.0,9.168021e+07
491,2019-12-16,Jual -4.0 saham,Beli 1.0 saham,Tahan,Tahan,56.0,9.224888e+07
492,2019-12-17,Tahan,Beli 1.0 saham,Tahan,Tahan,57.0,9.260938e+07


###7.4.2. Agen 2: PPO

In [174]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi5/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
490,2019-12-13,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
491,2019-12-16,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0
492,2019-12-17,Tahan,Tahan,Tahan,Tahan,0.0,100000000.0


###7.4.1. Agen 3: TD3

In [175]:
#Replace NaN value to 0
action_columns_td3=action_columns_td3.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_td3)

j=0
for j in range(numberOfRows):
  for i in action_columns_td3:
   val = action_columns_td3[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_td3, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi5/df_actions_and_account_str_value_td3(4).csv',index=False)
df_actions_str

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Beli 100.0 saham,Tahan,Tahan,200.0,1.000000e+08
1,2018-01-03,Tahan,Beli 12.0 saham,Tahan,Tahan,212.0,1.008074e+08
2,2018-01-04,Jual -100.0 saham,Beli 48.0 saham,Tahan,Tahan,160.0,1.020740e+08
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,160.0,1.011810e+08
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,160.0,1.034794e+08
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Tahan,Tahan,Tahan,160.0,1.078058e+08
490,2019-12-13,Tahan,Tahan,Tahan,Tahan,160.0,1.095668e+08
491,2019-12-16,Tahan,Tahan,Tahan,Tahan,160.0,1.113279e+08
492,2019-12-17,Tahan,Tahan,Tahan,Tahan,160.0,1.123845e+08


###7.4.1. Agen 4: SAC

In [176]:
#Replace NaN value to 0
action_columns_sac=action_columns_sac.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_sac)

j=0
for j in range(numberOfRows):
  for i in action_columns_sac:
   val = action_columns_sac[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_sac, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi5/df_actions_and_account_str_value_sac(4).csv',index=False)
df_actions_str

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Tahan,Beli 35.0 saham,35.0,1.000000e+08
1,2018-01-03,Beli 5.0 saham,Tahan,Tahan,Tahan,40.0,9.668643e+07
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,40.0,9.936975e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,40.0,1.034928e+08
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,40.0,1.042827e+08
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Tahan,Tahan,Tahan,40.0,6.537100e+07
490,2019-12-13,Tahan,Tahan,Tahan,Tahan,40.0,6.768814e+07
491,2019-12-16,Tahan,Tahan,Tahan,Tahan,40.0,7.031104e+07
492,2019-12-17,Tahan,Tahan,Tahan,Tahan,40.0,7.147625e+07


###7.4.1. Agen 5: DDPG

In [177]:
#Replace NaN value to 0
action_columns_ddpg=action_columns_ddpg.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ddpg)

j=0
for j in range(numberOfRows):
  for i in action_columns_ddpg:
   val = action_columns_ddpg[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ddpg, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi5/df_actions_and_account_str_value_ddpg(4).csv',index=False)
df_actions_str

,date,INCO.JK,INDF.JK,INTP.JK,UNTR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Beli 100.0 saham,Tahan,Tahan,200.0,1.000000e+08
1,2018-01-03,Tahan,Beli 12.0 saham,Tahan,Tahan,212.0,1.008074e+08
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,212.0,1.020740e+08
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,212.0,1.022088e+08
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,212.0,1.038177e+08
...,...,...,...,...,...,...,...
489,2019-12-12,Tahan,Tahan,Tahan,Tahan,212.0,1.087285e+08
490,2019-12-13,Tahan,Tahan,Tahan,Tahan,212.0,1.112492e+08
491,2019-12-16,Tahan,Tahan,Tahan,Tahan,212.0,1.136708e+08
492,2019-12-17,Tahan,Tahan,Tahan,Tahan,212.0,1.144105e+08


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.1'></a>
###7.5.1. Agen 1: A2C

In [178]:
#drop some columns
df_actions_and_account_value_a2c = df_actions_and_account_value_a2c.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_a2c:
    df_temp = df_actions_and_account_value_a2c[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic-1:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi5/1_A2C/df_states_and_actions_a2c_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,0.0
13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,0.0
13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,0.0
13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0
13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,4.0
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,-4.0
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,12.0
13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,12.0
13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,12.0
13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,12.0
13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,1.0
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,-22.0
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,1.0
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,1.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0
13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0
13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0
13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0
13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,1.0
13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,1.0
13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,1.0
13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,1.0
13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0
15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0
15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0
15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0


####String Action Version

In [179]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  #i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi5/1_A2C/df_states_and_actions_str_a2c_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,INCO.JK
0,13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,0.0,Tahan
1,13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,0.0,Tahan
2,13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,0.0,Tahan
3,13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0,Tahan
4,13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0,Tahan
490,15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,4.0,Beli 4.0 saham
491,15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,-4.0,Tahan
492,15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,INDF.JK
0,13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,12.0,Beli 12.0 saham
1,13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,12.0,Beli 12.0 saham
2,13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,12.0,Beli 12.0 saham
3,13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,12.0,Beli 12.0 saham
4,13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,-15.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,1.0,Beli 1.0 saham
490,15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,-22.0,Tahan
491,15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,1.0,Beli 1.0 saham
492,15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,1.0,Beli 1.0 saham


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,INTP.JK
0,13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0,Tahan
1,13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0,Tahan
2,13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0,Tahan
3,13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0,Tahan
4,13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0,Tahan
490,15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0,Tahan
491,15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0,Tahan
492,15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,UNTR.JK
0,13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,1.0,Beli 1.0 saham
1,13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,1.0,Beli 1.0 saham
2,13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,1.0,Beli 1.0 saham
3,13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,1.0,Beli 1.0 saham
4,13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,-4.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0,Tahan
490,15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0,Tahan
491,15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0,Tahan
492,15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0,Tahan


<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [180]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi5/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,0.0
13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,0.0
13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,0.0
13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0
13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,0.0
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,0.0
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,0.0
13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,0.0
13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,0.0
13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,0.0
13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,0.0
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,0.0
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,0.0
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0
13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0
13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0
13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0
13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,0.0
13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,0.0
13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,0.0
13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,0.0
13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0
15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0
15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0
15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0


####String Action Version

In [181]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi5/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INCO.JK
0,13136,2018-01-02,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,Tahan
1,13140,2018-01-03,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,Tahan
2,13144,2018-01-04,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,Tahan
3,13148,2018-01-05,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,Tahan
4,13160,2018-01-08,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,Tahan
490,15976,2019-12-13,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,Tahan
491,15988,2019-12-16,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,Tahan
492,15992,2019-12-17,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INDF.JK
0,13137,2018-01-02,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,Tahan
1,13141,2018-01-03,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,Tahan
2,13145,2018-01-04,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,Tahan
3,13149,2018-01-05,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,Tahan
4,13161,2018-01-08,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,Tahan
490,15977,2019-12-13,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,Tahan
491,15989,2019-12-16,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,Tahan
492,15993,2019-12-17,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INTP.JK
0,13138,2018-01-02,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,Tahan
1,13142,2018-01-03,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,Tahan
2,13146,2018-01-04,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,Tahan
3,13150,2018-01-05,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,Tahan
4,13162,2018-01-08,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,Tahan
490,15978,2019-12-13,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,Tahan
491,15990,2019-12-16,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,Tahan
492,15994,2019-12-17,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNTR.JK
0,13139,2018-01-02,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,Tahan
1,13143,2018-01-03,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,Tahan
2,13147,2018-01-04,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,Tahan
3,13151,2018-01-05,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,Tahan
4,13163,2018-01-08,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,Tahan
490,15979,2019-12-13,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,Tahan
491,15991,2019-12-16,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,Tahan
492,15995,2019-12-17,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,Tahan


<a id='6.4.3'></a>
###7.5.3. Agen 3: TD3

In [182]:
#drop some columns
df_actions_and_account_value_td3 = df_actions_and_account_value_td3.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_td3:
    df_temp = df_actions_and_account_value_td3[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi5/3_TD3/df_states_and_actions_td3_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,100.0
13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,0.0
13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,-100.0
13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0
13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,0.0
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,0.0
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,100.0
13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,12.0
13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,48.0
13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,0.0
13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,0.0
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,0.0
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,0.0
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0
13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0
13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0
13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0
13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,0.0
13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,0.0
13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,0.0
13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,0.0
13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0
15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0
15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0
15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0


####String Action Version

In [183]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi5/3_TD3/df_states_and_actions_str_td3_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INCO.JK
0,13136,2018-01-02,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,Beli 100.0 saham
1,13140,2018-01-03,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,Tahan
2,13144,2018-01-04,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,Tahan
3,13148,2018-01-05,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,Tahan
4,13160,2018-01-08,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,Tahan
490,15976,2019-12-13,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,Tahan
491,15988,2019-12-16,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,Tahan
492,15992,2019-12-17,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INDF.JK
0,13137,2018-01-02,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,Beli 100.0 saham
1,13141,2018-01-03,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,Beli 12.0 saham
2,13145,2018-01-04,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,Beli 48.0 saham
3,13149,2018-01-05,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,Tahan
4,13161,2018-01-08,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,Tahan
490,15977,2019-12-13,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,Tahan
491,15989,2019-12-16,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,Tahan
492,15993,2019-12-17,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INTP.JK
0,13138,2018-01-02,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,Tahan
1,13142,2018-01-03,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,Tahan
2,13146,2018-01-04,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,Tahan
3,13150,2018-01-05,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,Tahan
4,13162,2018-01-08,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,Tahan
490,15978,2019-12-13,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,Tahan
491,15990,2019-12-16,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,Tahan
492,15994,2019-12-17,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNTR.JK
0,13139,2018-01-02,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,Tahan
1,13143,2018-01-03,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,Tahan
2,13147,2018-01-04,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,Tahan
3,13151,2018-01-05,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,Tahan
4,13163,2018-01-08,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,Tahan
490,15979,2019-12-13,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,Tahan
491,15991,2019-12-16,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,Tahan
492,15995,2019-12-17,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,Tahan


<a id='6.4.4'></a>
###7.5.4. Agen 4: SAC

In [184]:
#drop some columns
df_actions_and_account_value_sac = df_actions_and_account_value_sac.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_sac:
    df_temp = df_actions_and_account_value_sac[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi5/4_SAC/df_states_and_actions_sac_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,0.0
13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,5.0
13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,0.0
13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0
13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,0.0
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,0.0
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,0.0
13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,0.0
13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,0.0
13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,0.0
13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,0.0
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,0.0
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,0.0
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0
13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0
13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0
13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0
13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,35.0
13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,0.0
13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,0.0
13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,0.0
13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0
15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0
15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0
15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0


####String Action Version

In [185]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi5/4_SAC/df_states_and_actions_str_sac_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INCO.JK
0,13136,2018-01-02,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,Tahan
1,13140,2018-01-03,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,Beli 5.0 saham
2,13144,2018-01-04,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,Tahan
3,13148,2018-01-05,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,Tahan
4,13160,2018-01-08,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,Tahan
490,15976,2019-12-13,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,Tahan
491,15988,2019-12-16,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,Tahan
492,15992,2019-12-17,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INDF.JK
0,13137,2018-01-02,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,Tahan
1,13141,2018-01-03,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,Tahan
2,13145,2018-01-04,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,Tahan
3,13149,2018-01-05,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,Tahan
4,13161,2018-01-08,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,Tahan
490,15977,2019-12-13,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,Tahan
491,15989,2019-12-16,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,Tahan
492,15993,2019-12-17,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INTP.JK
0,13138,2018-01-02,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,Tahan
1,13142,2018-01-03,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,Tahan
2,13146,2018-01-04,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,Tahan
3,13150,2018-01-05,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,Tahan
4,13162,2018-01-08,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,Tahan
490,15978,2019-12-13,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,Tahan
491,15990,2019-12-16,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,Tahan
492,15994,2019-12-17,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNTR.JK
0,13139,2018-01-02,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,Beli 35.0 saham
1,13143,2018-01-03,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,Tahan
2,13147,2018-01-04,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,Tahan
3,13151,2018-01-05,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,Tahan
4,13163,2018-01-08,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,Tahan
490,15979,2019-12-13,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,Tahan
491,15991,2019-12-16,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,Tahan
492,15995,2019-12-17,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,Tahan


<a id='6.5.5'></a>
###7.4.5. Agen 5: DDPG

In [186]:
#drop some columns
df_actions_and_account_value_ddpg = df_actions_and_account_value_ddpg.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ddpg:
    df_temp = df_actions_and_account_value_ddpg[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi5/5_DDPG/df_states_and_actions_ddpg_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,INCO.JK,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,100.0
13140,2018-01-03,INCO.JK,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,0.0
13144,2018-01-04,INCO.JK,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,0.0
13148,2018-01-05,INCO.JK,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,0.0
13160,2018-01-08,INCO.JK,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,INCO.JK,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,0.0
15976,2019-12-13,INCO.JK,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,0.0
15988,2019-12-16,INCO.JK,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,0.0
15992,2019-12-17,INCO.JK,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,INDF.JK,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,100.0
13141,2018-01-03,INDF.JK,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,12.0
13145,2018-01-04,INDF.JK,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,0.0
13149,2018-01-05,INDF.JK,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,0.0
13161,2018-01-08,INDF.JK,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,INDF.JK,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,0.0
15977,2019-12-13,INDF.JK,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,0.0
15989,2019-12-16,INDF.JK,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,0.0
15993,2019-12-17,INDF.JK,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,INTP.JK,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,0.0
13142,2018-01-03,INTP.JK,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,0.0
13146,2018-01-04,INTP.JK,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,0.0
13150,2018-01-05,INTP.JK,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,0.0
13162,2018-01-08,INTP.JK,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,INTP.JK,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,0.0
15978,2019-12-13,INTP.JK,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,0.0
15990,2019-12-16,INTP.JK,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,0.0
15994,2019-12-17,INTP.JK,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNTR.JK,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,0.0
13143,2018-01-03,UNTR.JK,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,0.0
13147,2018-01-04,UNTR.JK,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,0.0
13151,2018-01-05,UNTR.JK,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,0.0
13163,2018-01-08,UNTR.JK,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNTR.JK,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,0.0
15979,2019-12-13,UNTR.JK,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,0.0
15991,2019-12-16,UNTR.JK,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,0.0
15995,2019-12-17,UNTR.JK,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,0.0


####String Action Version

In [187]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi5/5_DDPG/df_states_and_actions_str_ddpg_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INCO.JK
0,13136,2018-01-02,2920.0,3040.0,2920.0,3001.898438,14717300.0,1.0,14.154749,56.639747,165.346926,9.77,13.799092,Beli 100.0 saham
1,13140,2018-01-03,3050.0,3110.0,3040.0,3021.712891,19792900.0,2.0,27.434247,57.259786,215.036026,9.15,4.300817,Tahan
2,13144,2018-01-04,3040.0,3120.0,3000.0,3081.156250,7036200.0,3.0,42.267691,59.075932,203.245831,9.22,2.062962,Tahan
3,13148,2018-01-05,3100.0,3220.0,3100.0,3140.599854,12045900.0,4.0,58.149593,60.799127,240.813122,9.22,4.171605,Tahan
4,13160,2018-01-08,3160.0,3170.0,3070.0,3140.599854,10755100.0,0.0,69.930004,60.799127,195.721607,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15972,2019-12-12,3330.0,3400.0,3330.0,3328.837646,17337900.0,3.0,-59.990880,49.955000,20.279905,13.94,5.727264,Tahan
490,15976,2019-12-13,3400.0,3510.0,3380.0,3457.632080,20040100.0,4.0,-33.286936,53.131860,68.126086,12.63,4.966053,Tahan
491,15988,2019-12-16,3510.0,3610.0,3500.0,3576.519043,16947000.0,0.0,-2.501845,55.810515,121.090136,12.14,5.562965,Tahan
492,15992,2019-12-17,3620.0,3670.0,3540.0,3576.519043,15492200.0,1.0,21.646040,55.810515,135.814337,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INDF.JK
0,13137,2018-01-02,7650.0,7700.0,7550.0,6197.431152,6875700.0,1.0,-14.553107,43.053756,30.342660,9.77,13.799092,Beli 100.0 saham
1,13141,2018-01-03,7600.0,7700.0,7600.0,6279.516602,11532300.0,2.0,-9.668867,46.124584,56.920960,9.15,4.300817,Beli 12.0 saham
2,13145,2018-01-04,7650.0,7725.0,7600.0,6341.081055,6861700.0,3.0,-0.820872,48.288137,77.351861,9.22,2.062962,Tahan
3,13149,2018-01-05,7700.0,7700.0,7625.0,6300.038086,8060200.0,4.0,2.846589,46.986814,71.224161,9.22,4.171605,Tahan
4,13161,2018-01-08,7700.0,7875.0,7700.0,6443.687988,13357200.0,0.0,17.146768,51.699674,146.091021,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15973,2019-12-12,7850.0,7875.0,7625.0,6714.087891,11772400.0,3.0,10.111270,46.754614,-146.147903,13.94,5.727264,Tahan
490,15977,2019-12-13,7775.0,7775.0,7650.0,6824.155273,5810400.0,4.0,0.100794,49.465576,-139.708205,12.63,4.966053,Tahan
491,15989,2019-12-16,7750.0,7875.0,7750.0,6934.222168,4859200.0,0.0,1.036944,51.994047,-41.452806,12.14,5.562965,Tahan
492,15993,2019-12-17,7900.0,7950.0,7850.0,7000.262207,6208100.0,1.0,7.026730,53.440010,38.317047,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,INTP.JK
0,13138,2018-01-02,21950.0,23000.0,21350.0,18384.472656,1017900.0,1.0,471.242145,62.624360,199.526927,9.77,13.799092,Tahan
1,13142,2018-01-03,22800.0,23000.0,22650.0,18384.472656,2208100.0,2.0,556.323866,62.624360,202.851260,9.15,4.300817,Tahan
2,13146,2018-01-04,23000.0,23000.0,22375.0,18384.472656,1859200.0,3.0,616.643461,62.624360,173.635266,9.22,2.062962,Tahan
3,13150,2018-01-05,22625.0,23225.0,22450.0,18564.318359,1501100.0,4.0,671.221793,63.402018,165.206347,9.22,4.171605,Tahan
4,13162,2018-01-08,23225.0,23400.0,22725.0,18584.302734,1069100.0,0.0,707.927538,63.489342,156.657516,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15974,2019-12-12,19525.0,19750.0,19400.0,16671.226562,2210900.0,3.0,-6.877867,46.801780,-105.784074,13.94,5.727264,Tahan
490,15978,2019-12-13,19600.0,20000.0,19500.0,17098.693359,1986400.0,4.0,-7.054024,49.534574,-48.870007,12.63,4.966053,Tahan
491,15990,2019-12-16,20000.0,20075.0,19725.0,16927.705078,1529200.0,0.0,-20.751742,48.503546,-37.730311,12.14,5.562965,Tahan
492,15994,2019-12-17,19950.0,20025.0,19725.0,17120.066406,2500300.0,1.0,-15.902021,49.721468,-27.012906,12.29,0.951754,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNTR.JK
0,13139,2018-01-02,35400.0,35400.0,34100.0,28027.521484,3701100.0,1.0,394.738176,53.290363,114.551211,9.77,13.799092,Tahan
1,13143,2018-01-03,34025.0,34100.0,32800.0,27145.250000,4653700.0,2.0,287.665093,49.675927,5.254329,9.15,4.300817,Tahan
2,13147,2018-01-04,33500.0,34000.0,33125.0,27904.414062,3575000.0,3.0,261.057788,52.541195,33.098164,9.22,2.062962,Tahan
3,13151,2018-01-05,33750.0,35425.0,33750.0,29073.939453,4634300.0,4.0,330.532063,56.489251,125.716240,9.22,4.171605,Tahan
4,13163,2018-01-08,35175.0,35800.0,35000.0,29299.634766,3940600.0,0.0,399.200913,57.200049,166.713443,9.52,4.209610,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,15975,2019-12-12,20800.0,20875.0,20450.0,18155.101562,4193900.0,3.0,-264.082615,44.730790,-129.278411,13.94,5.727264,Tahan
490,15979,2019-12-13,20750.0,21200.0,20700.0,18798.742188,4618500.0,4.0,-244.171944,47.969447,-85.881206,12.63,4.966053,Tahan
491,15991,2019-12-16,21250.0,22125.0,21200.0,19531.160156,4904500.0,0.0,-167.363324,51.326896,-18.063001,12.14,5.562965,Tahan
492,15995,2019-12-17,22000.0,22375.0,21750.0,19864.076172,4607100.0,1.0,-78.720954,52.760264,20.354842,12.29,0.951754,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [188]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.038418
Cumulative returns    -0.073922
Annual volatility      0.110917
Sharpe ratio          -0.298545
Calmar ratio          -0.184933
Stability              0.106454
Max drawdown          -0.207742
Omega ratio            0.949839
Sortino ratio         -0.420936
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.070907
Daily value at risk   -0.014106
dtype: float64


###8.1.2 Agen ppo

In [189]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.0
Cumulative returns     0.0
Annual volatility      0.0
Sharpe ratio           NaN
Calmar ratio           NaN
Stability              0.0
Max drawdown           0.0
Omega ratio            NaN
Sortino ratio          NaN
Skew                   NaN
Kurtosis               NaN
Tail ratio             NaN
Daily value at risk    0.0
dtype: float64


/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:713: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:799: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/usr/local/lib/python3.7/dist-packages/empyrical/stats.py:1528: RuntimeWarning: invalid value encountered in double_scalars
  np.abs(np.percentile(returns, 5))


###8.1.3 Agen TD3

In [190]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return          0.061369
Cumulative returns     0.123845
Annual volatility      0.280846
Sharpe ratio           0.352413
Calmar ratio           0.208073
Stability              0.202454
Max drawdown          -0.294942
Omega ratio            1.063619
Sortino ratio          0.519013
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.133268
Daily value at risk   -0.034991
dtype: float64


###8.1.4 Agen SAC

In [191]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.168725
Cumulative returns    -0.303897
Annual volatility      0.344382
Sharpe ratio          -0.365505
Calmar ratio          -0.359838
Stability              0.834444
Max drawdown          -0.468893
Omega ratio            0.939370
Sortino ratio         -0.511361
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.076142
Daily value at risk   -0.043888
dtype: float64


###8.1.5 Agen DDPG

In [192]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.063491
Cumulative returns     0.128253
Annual volatility      0.261000
Sharpe ratio           0.366492
Calmar ratio           0.233246
Stability              0.074873
Max drawdown          -0.272206
Omega ratio            1.063872
Sortino ratio          0.537513
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.255253
Daily value at risk   -0.032503
dtype: float64
